## Import All Required Libraries

In [27]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2
import requests

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [28]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Establish External Connections

In [29]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

## Load Initial Training Data

In [30]:
# load training data
new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[0:30]
new_batch.head()

,playlist_uri,playlist_name
1427,spotify:playlist:1j0yRynr9glAso7KRFyQUx,Clean Basketball Playlist
1455,spotify:playlist:1k6TQ5milpp60GdyYjzge6,abcdefu clean
1456,spotify:playlist:1k6YKXx2XFCujamPiKKmUd,Sad workout playlist ??
1457,spotify:playlist:1k7O0iZP3SITsTibnSmljj,Disco House 2022
1458,spotify:playlist:1KB3Nd6kXGoXtoj3Pwula4,HOUSE PARTY SLAPPERS


## Iterate Through Training Data to Change Grain of Data

In [31]:
load_batch = []
series = new_batch['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                load_batch.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                load_batch.append(tracks)
    except:
            pass
load_batch = pd.concat(load_batch)
if set(['track.album.is_playable','track.album.restrictions.reason', 'track.external_ids.spotify']).issubset(load_batch.columns):
        load_batch = load_batch.drop(columns=['track.album.is_playable', 'track.album.restrictions.reason', 'track.external_ids.spotify', 'track.is_playable'])
else:
        load_batch
load_batch.head()

100%|██████████| 30/30 [00:28<00:00,  1.04it/s]


,added_at,is_local,primary_color_x,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_external_urls_spotify,...,owner_href,owner_id,owner_type,owner_uri,tracks_href,tracks_limit,tracks_next,tracks_offset,tracks_previous,tracks_total
0,2022-02-19T18:37:04Z,False,None,https://open.spotify.com/user/31pdlqg6eo7gfyrk...,https://api.spotify.com/v1/users/31pdlqg6eo7gf...,31pdlqg6eo7gfyrksawbtpl4gb7e,user,spotify:user:31pdlqg6eo7gfyrksawbtpl4gb7e,single,https://open.spotify.com/album/7ktInDlvdB1fVBT...,...,https://api.spotify.com/v1/users/31pdlqg6eo7gf...,31pdlqg6eo7gfyrksawbtpl4gb7e,user,spotify:user:31pdlqg6eo7gfyrksawbtpl4gb7e,https://api.spotify.com/v1/playlists/1k6TQ5mil...,100,None,0,None,2
1,2022-02-19T18:38:18Z,False,None,https://open.spotify.com/user/31pdlqg6eo7gfyrk...,https://api.spotify.com/v1/users/31pdlqg6eo7gf...,31pdlqg6eo7gfyrksawbtpl4gb7e,user,spotify:user:31pdlqg6eo7gfyrksawbtpl4gb7e,single,https://open.spotify.com/album/0Zq00ywbUyDLVD8...,...,https://api.spotify.com/v1/users/31pdlqg6eo7gf...,31pdlqg6eo7gfyrksawbtpl4gb7e,user,spotify:user:31pdlqg6eo7gfyrksawbtpl4gb7e,https://api.spotify.com/v1/playlists/1k6TQ5mil...,100,None,0,None,2
0,2020-12-09T04:05:27Z,False,None,https://open.spotify.com/user/natemongeau,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,album,https://open.spotify.com/album/1KOmHyNLuOe5YrP...,...,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,https://api.spotify.com/v1/playlists/1k6YKXx2X...,100,https://api.spotify.com/v1/playlists/1k6YKXx2X...,0,None,166
1,2020-12-09T04:05:42Z,False,None,https://open.spotify.com/user/natemongeau,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,album,https://open.spotify.com/album/1KOmHyNLuOe5YrP...,...,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,https://api.spotify.com/v1/playlists/1k6YKXx2X...,100,https://api.spotify.com/v1/playlists/1k6YKXx2X...,0,None,166
2,2020-12-09T04:05:49Z,False,None,https://open.spotify.com/user/natemongeau,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,single,https://open.spotify.com/album/2bw00gRKNKbTFOq...,...,https://api.spotify.com/v1/users/natemongeau,natemongeau,user,spotify:user:natemongeau,https://api.spotify.com/v1/playlists/1k6YKXx2X...,100,https://api.spotify.com/v1/playlists/1k6YKXx2X...,0,None,166


## Upload Training Data to playlist_tracks Table

In [32]:
load_batch.to_sql('playlist_tracks', engine, if_exists='append')

855

## Identify All Artists in Every Song

In [33]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM artists''', engine)
outer = load_batch.merge(df, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

 20%|██        | 512/2510 [01:02<03:51,  8.65it/s]Expected id of type track but found type A%24AP+Rocky+-+Fuckin%27+Problems+ft+Xavier+Dunn+%28Vijay+%26+Sofia+Zlatko%2C+Kasual+Remix%29 spotify:local:::A%24AP+Rocky+-+Fuckin%27+Problems+ft+Xavier+Dunn+%28Vijay+%26+Sofia+Zlatko%2C+Kasual+Remix%29:348
HTTP Error for GET to https://api.spotify.com/v1/tracks/348 with Params: {'market': None} returned 400 due to invalid id
Expected id of type track but found type Fuckin%27+Problems+-+A%24AP+Rocky+%28Xavier+Dunn+Cover+%235%29 spotify:local:::Fuckin%27+Problems+-+A%24AP+Rocky+%28Xavier+Dunn+Cover+%235%29:283
HTTP Error for GET to https://api.spotify.com/v1/tracks/283 with Params: {'market': None} returned 400 due to invalid id
 99%|█████████▉| 2497/2510 [05:04<00:01,  8.01it/s]

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,Mustafa Sandal,spotify:track:1aFuXmfz6bYOvUDOLZiqys,1.0
0,Eylem,spotify:track:1TCQIEgq3QU7o5dXaRnePe,1.0
0,Hepsi,spotify:track:3CDEGpKHri2sJCHUOcj7cY,1.0
0,Grup Laçin,spotify:track:7GDPQinj3BOwo2vWqTdsTn,1.0
0,Sibel Can,spotify:track:3Kug9xmrzcUs4AXQQl6klM,1.0
...,...,...,...
0,Drake,spotify:track:5iUaAkgZnxOEUI2BzsE5bM,1.0
1,Lil Wayne,spotify:track:5iUaAkgZnxOEUI2BzsE5bM,2.0
0,Drake,spotify:track:1iDIWccY2uqjp9ZNYaMMMP,1.0
0,Drake,spotify:track:3wxt7fCP1oNQrBTIeJ6QZA,1.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
df2

RN,track_uri,artist1,artist2,artist3,artist4,artist5
0,spotify:track:00HH7QLCCOZb061uzLiTnk,Flores Harmonici,Hildegard von Bingen,Unknown Artist,NaN,NaN
1,spotify:track:00bq0YdeIHvPlsG657Elj7,Tyler Wood,NaN,NaN,NaN,NaN
2,spotify:track:01BCvCKQNDmFQQQSjrzPnm,Noyz Narcos,Salmo,NaN,NaN,NaN
3,spotify:track:01SyAjCtqtgNmrBmvSJBV9,Cem Karaca,NaN,NaN,NaN,NaN
4,spotify:track:01YmDEvbiVb5q5eCGNlB6U,Jenni Potts,ODESZA,NaN,NaN,NaN
...,...,...,...,...,...,...
2625,spotify:track:7z97Vh2SJ7OKdO4lON8RyK,Cody Johnson,NaN,NaN,NaN,NaN
2626,spotify:track:7zJa2cpFsYGWQfZ9xz1nYg,Cody Johnson,NaN,NaN,NaN,NaN
2627,spotify:track:7zL8clqEZfzJWEFO0cNj8B,Steven C,NaN,NaN,NaN,NaN
2628,spotify:track:7zdX4EIcnfbs6b5ZYkJeb7,Bloodwitch,NaN,NaN,NaN,NaN


## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

630

## Get All Audio Features

In [ ]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM audio_features''', engine)
outer = load_batch.merge(df, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'])

base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)

100%|██████████| 2651/2651 [04:13<00:00, 10.46it/s]


## Upload Audio Features to Database

In [ ]:
df3.to_sql('audio_features', con=engine, if_exists='append')

651

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT playlist_uri, playlist_name, owner_uri, owner_display_name, collaborative, description, followers_total,
af.track_uri, track_name, artist1, artist2, artist3, artist4, artist5, album_uri, album_name, release_date, 
added_by_uri, added_at, track_album_total_tracks, af.duration_ms, "isExplicit", track_popularity, 
track_preview_url, video_thumbnail_url, danceability, energy, loudness, key, mode, acousticness, speechiness, 
instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN artists a ON pt.track_uri = a.track_uri
JOIN audio_features af ON pt.track_uri = af.track_uri;''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+", index=False)